In [1]:
%load_ext Cython 
import numpy as np
import random, time
from multiprocessing import Pool, freeze_support
from io import StringIO
import itertools,time
import matplotlib.widgets as widgets
%pylab inline
def linlin(x, smi, sma, dmi, dma): return (x-smi)/(sma-smi)*(dma-dmi)+dmi
from threading import Thread

# Audio server
from pyo import *
# Is 1024 too large for the bufferSize in this case. 
fs = 44100/2
s = Server(sr=fs, nchnls=2, buffersize=2048, duplex=0).boot()
s.start()
# s.stop()

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [5]:
# Class for playing the audio 
class playAudioNoThread:       
    def __init__(self, sampling_rate, dataLen = 1.0,  play_back_rate = 1.0, bs = 44100, amp = 0.5):
#         audioVec = np.zeros(bufferSize)
        self.sf = sampling_rate
        self.rate = play_back_rate
        self.amp = amp
        self.bs = int(bs)
        self.dataLen = dataLen
        self.t = DataTable(size =self.bs, chnls=1)  # Data table
        self.c = Osc(table=self.t, freq=[ 1.0/self.dataLen * self.rate, \
                                         1.0 / self.dataLen * self.rate],\
                     mul=self.amp).out() # Table playback. 

    def looping(self, audioVec):
        # Continous mode
        self.t.replace(list(audioVec))
        self.c.setTable(self.t)
    
    def single(self, audioVec):
        # Currently no add silence. Need to do so. 
        self.t = DataTable(size =self.bs, chnls=1)  # Data table
        self.c = Osc(table=self.t, freq=[ 1.0/self.dataLen * self.rate, \
                                         1.0 / self.dataLen * self.rate],\
                     mul=self.amp).out() # Table playback. 
        self.t.replace(list(audioVec))
        self.c.setTable(self.t)
        time.sleep(self.dataLen/self.rate)
        self.c.stop()
        
    def stopAudio(self):
        self.c.stop()
        
# Create a rigid window to ramp on/off the audio vector to prevent clipping. 
# But I still hear clipping. 
def makeWindow(s, rampUp = 0.02, rampDown = 0.02):
    w = np.append(np.linspace(0, 1.0, num = int(s * rampUp)), \
      np.ones(s - int(s * rampUp)- int(s * rampDown)))
    w = np.append(w, np.linspace(1.0, 0, num = int(s * rampDown)))
    return w

# Add some silence to the end
def addSilence(av, bs = 5000):
    return np.append(av, np.zeros(0.2 * bs))

In [6]:

# Turn list into integer
def list2int(numbers):
    return int(''.join([ "%d"%x for x in numbers]))

# Slider update function, for the interaction
def sliderUpdate(val):
    global dt, resistant
    sigma = ssigma.val
    exp_table = updateExpTable(sigma, delta_dis)
    dt = sdt.val
    resistant = sresistant.val
    
def sigmaUpdate(val):
    global sigma, exp_table, delta_dis
    sigma = ssigma.val
    exp_table = updateExpTable(sigma, delta_dis)
    
# Create an exponential table for the lookup calculation of potential. 
def createExpTable( dim, sigma, exp_resolution = 1000000 ):
    delta_dis = np.linspace(0.0, dim, num = exp_resolution) # Range of distance difference. 
    tab = updateExpTable(sigma, delta_dis)
    return tab, dim

def updateExpTable(sigma, delta_dis):
    sigma2 = sigma * sigma
    return exp(- delta_dis/sigma2)/sigma2